In [28]:
import pandas as pd
import numpy as np

import datetime
from datetime import timedelta

Функция для скачивания данных

In [15]:
import os
import pandas as pd

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Task 1.2

In [4]:
df = pd.read_csv('data/2022-04-01T12_df_sales_detail.csv')

In [14]:
df.groupby('good', as_index=False).price.sum().sort_values('price', ascending=False).head(3)

,good,price
3,chefs pizza,24558240
6,double pepperoni pizza,22558380
5,chicken bbq pizza,17622540


# Task 1.3

Оцените средний срок, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

In [16]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

In [56]:
df_sales= read_database('2022-04-01T12_df_sales.csv')

df_logs = read_database('2022-04-01T12_df_web_logs.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_logs['date'] = pd.to_datetime(df_logs['date'])

# объединим продажи и логи
df_sales_w_logs = df_sales.merge(df_logs, how='inner', on='user_id', suffixes=('_sale', '_log'))
# оставим только логи не раньше чем за 2 часа до продажи
df_sales_w_logs = df_sales_w_logs.loc[df_sales_w_logs['date_sale'] - df_sales_w_logs['date_log'] <= timedelta(hours=2), :]
# для каждой продажи выделим первый лог
df_first_page = df_sales_w_logs.groupby(['sale_id', 'date_sale'], as_index=False)['date_log'].min()
# посчитаем разницу в минутах
diff = (df_first_page['date_sale'] - df_first_page['date_log']).dt.total_seconds() / 60
# посчитаем среднее
np.round(diff.mean())


17.0

# Task 1.4

Интересно, как много пользователей возвращается к нам из месяца в месяц. Какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

In [83]:
febr_users = df_sales.loc[(df_sales['date'].dt.month == 2), 'user_id'].drop_duplicates().copy()
march_users = df_sales.loc[(df_sales['date'].dt.month == 3), 'user_id'].drop_duplicates().copy()
march_users = march_users.to_frame()
march_users['return'] = 1

retention = febr_users.to_frame().merge(march_users, how='left', on='user_id').fillna(0)
np.round(retention['return'].mean(), 2)


0.66

In [84]:
df_sales

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468
...,...,...,...,...,...,...
203842,1203843,2022-04-01 11:59:15,2,1,1410,7d3869
203843,1203844,2022-04-01 11:59:20,1,0,780,b7feae
203844,1203845,2022-04-01 11:59:43,1,0,600,7cdcc7
203845,1203846,2022-04-01 11:59:45,1,1,630,47b825


In [85]:
df_sales.loc[df_sales['date'] >= None]

TypeError: Invalid comparison between dtype=datetime64[ns] and NoneType

In [92]:
a = 2
if a is None:
    print('None')
else:
    print(a)

2
